# Lab2.2 From Text to Linguistic Features

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

This notebook describes how different kinds of implicit and explicit features of texts can be represented as so-called *one-hot-vectors* and combined in a single representation of a text. At the end of this notebook, you learned:

- How text is annotated with implicit and explicit features
- How to create *one-hot-encodings* in vector representations for combining diverse textual features
- How to train a *Named Entity Recognition and Classification* (NERC) classifier with combinations of one-hot-encodings

**Background reading:**

NLTK Book
Chapter 7, section 5: https://www.nltk.org/book/ch07.html

## 1. Natural language text as features

### 1.1 Text as a bundle of features

A text consists of a sequence of words on which we impose syntax and semantics. A machine needs to learn to associate the structural properties of the text to some interpretation. We can use various properties to do this:

- the words (regardless of the order or in order)
- the words and their frequency
- the part-of-speech of words
- grammatical relations such as dependencies
- word pairs, sequences of three words, four words, etc. (so-called word n-grams)
- the characters that make up the words (so-called character n-grams)
- sentences with words
- phrases
- the meaning of words
- the meaning of combinations of words
- word length, sentence length
- word position in a text
- discourse structure: title, header, caption, body, conclusion sections
- etc....

Some of the above properties, we get for free if we split a text into tokens (the words), e.g. by using spaces. Still, we need to consider what to do with punctuation and how to treate upper/lower cases (the word shape). Other properties are not explicit, such as the part-of-speech of words, phrases, syntax and the meaning.

### 1.2 Annotation of text

In order to make implicit properties of a text explicit to a machine, people need to annotate a text. In Natural Language Processing, annotated texts play a crucial role. Annotations are used as features for training a system in addition to just the words or they represent the labels (the interpretations of the people) that the system needs to predict. Likeiwse, we can distinguish two types of annotations:

<ol>
<li> *Human interpretation*: Labels that represent the interpretation of a text that a machine is supposed to learn and predict
<li> *Machine annotation*: Labels that a machine assigns to a text either through rules or by learning this from the annotations of type *1* by people.
</ol>

You can imagine that a machine *A* that is trained to assign labels can help another machine *B* to learn to interpret text that is enriched with labels from machine *A*. This creates a pipeline of dependent machines:

* Human annotators assign part-of-speech tags to texts
* Machine *A* learns to assign part-of-speech tags to text
* Human annotators assign entity mentions to text
* Machine *B* learns to assign entity phrases to text that is enriched with part-of-speech tags from machine *A*

Try to think of how you can build complex pipelines with machines helping each other.

We next focus on the first human annotation process which is at the basis of all.

## 1.3 The Human Annotation process

Annotation is a delicate process during which all kinds of decisions are made that impact the performance of systems and the emprical value of the evaluation. The process consists roughly of the following steps:

<ol>
<li> Collect texts: e.g. tweets, news, blogs, books
<li> Define an annotation scheme or code book which explains:
    <ol>
<li>  the tag set or set of labels (e.g. PoS labels, emotions, entity types)
<li>  the unit of the annotation: word, phrase, sentence, paragraph, document
<li>  the criteria to apply a tag to a piece of text
    </ol>
<li>  Train human annotators to use the annotation scheme (or create a crowd task)
<li>  Provide an annotation tool that loads texts and allows the annotator to assign tags
<li>  Store the annotations with the text in some structured output format representing the text and the annotations
<li>  Determine the Inter-Annotator-Agreement (IAA) by analysing texts annotated by at least two annotators
<li>  Fix disagreements (adjudication): if IAA is too low (e.g. less than 60 Kappa score) the task is considered too difficult or too unclear
</ol>

The IAA is often considered the upper ceiling of NLP. Ask yourself the question: can machines do better than humans and how would you measure this?

A machine that tries to learn from the human annotations can use any type of text properties. In most cases it uses the words but it can also use the part-of-speech of the word, the syntactic structure, semantic properties, and so on, and so on. So let's see how these annotations are usually represented in NLP.

### 1.4 A common representation format for text annotations

It is not trivial to represent annotated data in a way that we can easily train NLP systems as well as evaluate them. One of the most used formats is called [Inside–Outside–Beginning (IOB)](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)), which was typically designed to label phrases. In this annotation, we indicate for each word token in a sequence of text whether it belongs to some type of *interpretation* (represented by a label) or not, e.g. being part of a noun phrase or not. Furthermore if a token is part of an *interpretation*, we can also mark it as being inside or at the beginning of the phrase. 

The IOB format was designed and promoted by the Conference of Natural Language Learning ([CoNLL](https://www.conll.org)) which regularly organises NLP challenges or tasks for researchers. They provide the training and test data and the evaluation metrics to measure system performance.

One of the most popular datasets with IOB annotation is [CoNLL-2003](http://aclweb.org/anthology/W03-0419), which is among others used for the task of Named-Entity Recognition and Classification (NERC). NERC is considered considered as a special case of phrase structure annotation, i.e. detecting the sequence of words that can be interpreted as the name of an entity. On top of that, systems need to decide what type of named-entity phrase we are dealing with. Common types are: PERSON, LOCATION and ORGANISATION, but also others are used.

Let's consider an example of a text fragment in the IOB annotation:

```... Germany's representative to the European Union ...```

In the IOB-format, each token is represented on a separate line and the annotations are added separated by tabs into columns:

```
Germany NNP B-NP B-LOC
's  POS B-NP O
representative NN I-NP O
to TO B-PP O
the DT B-NP O
European NNP I-NP B-ORG
Union NNP I-NP I-ORG
```

The first observation is that all information is represented at the **token-level**. For each token, e.g., *Germany*, we receive information about:

* **the word**: e.g., *Germany*
* **the part of speech**: e.g., *NNP* (from [Penn Treebank](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html))
* **the phrase type**: e.g., a noun phrase
* **the NERC label**: e.g., a location (LOC), an organisation (ORG).

This example contains two named entities: *Germany* and *European Union*. *Germany* is of the type location (LOC) and the *European Union* is an organization (ORG). Every first token of a named entity is then prefixed with *B-*. Every token after that, e.g., *Union* in *European Union*, is prefixed with *I-*.

The task for a system is to decide 1) which tokens are part of a named-entity expressions and which words are not and 2) what the type is of this entity expression.

Please note that the IOB format is at the **token-level**, which means that we also are going to train and evaluate an NLP system at the token-level! The goal will hence not be to classify *European Union* as an *Organization*, but to classify:
* *European* as the first token of an entity that is an *Organization*
* *Union* as a token inside of an entity that is an *Organization*

Please make sure you understand the format before you proceed ;)

The annotation in IOB style can be quite rich as is shown in the next example in which many columns are given for each token providing many different features.

![title](images/iob-example.png)

The first column in this example lists the identifier for each token. The second is the token itself, followed by the lemma, two columns with part-of-speech tags (POS), some other feature, the token identifier that is the head of the phrase to which a token belongs ("museum" token 2 is the head of "0" The and "1" American), the syntactic dependency relation and the dependency relation combined. The final column indicates again the named-entity expressions and the type.

The token features are usually used as additional features to help a classifier learn when a IOB tag applies. Instead of just the words, the classifier can learn that words with certain part-of-speech, syntactic dependencies are more or less likely to be associated with certain tags. Representing more features for tokens, thus can help improving the precision of the classifier as it has more knowledge to combine.

More features can also increase the recall and coverage. If the unseen data contains words that have not been seen in the training data, it can still use the other features of this unkown words to make a prediction.

Typically, data sets are created for a specific task. The above example is taken from a dataset for NERC. This usually means that the IOB tags for the NERC task are added by people as the gold standard label. The other features: tokens, lemmas, part-of-speech, dependencies, are typically generated by software (e.g. trained on other data). For example, a tokenization programm will split the text into sentences and tokens, while a PoS tagger will lemmatise and assign the part-of-speech, and finally a syntactic parser will automatically add the constituent structures and dependencies. These auxiliary programs that generate explicit features for implicit propoperties are often also machine learning modules trained in yet other data. Typically, people only contribute the labels that form the predictions that the classifier is supposed to learn.

So, you need to realize that the data set has a mixture of gold annotations (added by people) and so-called copper (generated by machines) or silver (generated by machines and checked by humans) annoations.


**Why is this important?**

If you take a dataset *X* to train a system with rich features partly generated by machines and you want to test or apply it to dataset *Y* then you have to make sure that dataset *Y* gets the same rich feature representations, partly generated by machines. For example, if *X* uses different part-of-speech tags or interprets tokens differently in terms of part-of-speech than the systems used to represwent *Y*, we cannot expect the classifier to match the feature representations across the datasets and the performance will be low. Therefore, the features in the training and test set should be harmonized both in terms of labels and interpretations.

Knowing this, what do you think will happen if the PoS tagger is trained on another genre than the genre that is annotated for for example NERC?

Assuming that you will be able to generate IOB representations with many different features generated in the same way for both the training and test texts, the next question is:

### How can we transfer these IOB annotations to a uniform vector representation to build and apply a classifier?

The values for all these features are a mixture of words, symbols and numbers. So how can SVM or Skikitlearn deal with these diverse values in one single vector representation? We are going to address this in the next section.

## 2 One-hot-encoding of features

The answer to this question is the notion of a **one hot encoding**! In one-hot-encoding, you use a column for **each possible value** of a feature. Check again the IOB representation we have just seen. In IOB, each column represents a type of feature and we may have different values for each token in a specific column. In the *one hot encoding* you would have a range of columns, one for each value, that together make up one type of feature, e.g. part-of-speech. If a part-of-speech applies to a token, then the column reprensting that value will be scored with the value "1", whereas all the others remain "0".

We already used the principle for our *bag-of-words* representation of a text. The vocabulary of the training data was used to create one (possibly huge) vector in which each each position represents a single word. So the words of a specific sentence, a document or a tweet can be represented by this complete vector by putting a "1" in the slots of the words that it contains.

Now imagine we do the same for the part-of-speech of each token. In that case, we create a vector with all possible part-of-speech values that occur in the data. The next image shows some examples of representation for the 36 part-of-speech (PoS) tags taken from the Penn Treebank. The PoS vector thus has 36 columns for holding the values:

![title](images/one-hot-pos.png)


Looking at the above IOB example, we can imagine that we do this for all the columns given as a feature, such as the constituent a token is part of. Following this principle, each token in the above IOB annotation is represented by a combination of vectors:

* the vocabulary vector: all zeros except for the token itself (one-hot)
* the PoS vector: 35 zeros except for the PoS of the token (one-hot)
* the constituent vector: X Zeros except for the constituent it is part of (one-hot)
* the constituent head: again the vocabulary vector with all zeros except for the slot that represents the lemma of the head (one-hot)
* etc..

You can imagine that the vector representation of each token becomes very large (tens-of-thousands of positions), whereas most positions are zero. Not very efficient but very precise. Also note that some features are still rather complex and there are various options to represent them as vectors. These details will be discussed in the machine learning course.

Again, all feature values from the training set are represented in these vectors and any other text that is classified needs to be represented with the same features.

*Example*

In the next example, we will represent two token features **part of speech** and **lemma** as one-hot-encodings for the purpose of NERC.

Let's first generate those values for each of our tokens, with SpaCy. We use a very simple text sentence to illustrate the one-hot-encoding of features.

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

text="Germany's representative to the European Union"

doc=nlp(text)

Now we extract the part-of-speech and the lemma for each token to represent the token features as the training_instances:

In [ ]:
training_instances=[]
for token in doc:
    one_training_instance={'part-of-speech': token.pos_, 'lemma': token.lemma_} # this concatenates the PoS and Lemma
    print("THis is a training token instance:", one_training_instance)
    training_instances.append(one_training_instance)

In [ ]:
print('Number of tokens = ', len(training_instances))
print('The sequence of tokens is represented as a list of tuples:', training_instances)

Our instance information is now a list with dictionaries, with each dictionary representing a training instance (token). For each instance, we store two values: POS tag and lemma. Next, we will convert these values to a numeric vector representation, as *one-hot-encodings* using Scikit-learn.

### 2.1 Vectorizing our features

To accomplish this, we use the **DictVectorizer** function from sklearn ([link to documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html)). 

Please recall that in lab session 2 we used two other vectorizers, that create bag-of-words or tf-idf vectors from a vocabulary of words. Those two vectorizers operated on a vocabulary level: they assigned a non-zero value to the words which occur in an input, and zero to all others.

The DictVectorizer does something similar, but on a feature level: for each feature (let's say POS tag) DictVectorizer assigns a value of 1 to its value (for example, VERB) and zeros to all others (e.g., NOUN, PROPN, etc.). We use the function *fit_transform* and apply it to our training instances. Note that the formats of the data are important.

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
vec = DictVectorizer()

the_array = vec.fit_transform(training_instances).toarray() 
# The toarray() is only there for educational purposes. 
# Please do not use it in the assignment since you might get memory issues when applying this to large data sets

### 2.3 Analyzing the vectorized format

Let's now print the resulting vector representation. Each **row** represents the features for one token. Each **column** corresponds to one feature value (for example, VERB part-of-speech).

In [ ]:
print(the_array)

Generally speaking, each column represents a **specific value** of a lemma or POS tag. We can get the specific feature names for each column from the vectorizer:

In [ ]:
print(vec.get_feature_names_out())

We can see that the first seven slots of the vector represent the vocabulary and the last five slots represent the part-of-speech. Following the order of the *feature_names*, we can conclude that the second column for example stands for the lemma 'European'. Most words do not have this lemma, but the second last word has it. For that reason, we can see that the second column in the second last row has a value 1. All other rows have zeros in that column, because their lemma is different.

Similarly, the last column represents the tokens with a PROPN (proper noun) part-of-speech. We can see that three words have this part-of-speech tag, namely the words represented in the rows: 1, 6 and 7.


As a final analysis step, let's inspect the first row, i.e. the one hot encoding representation of the following training instance:
```
{'part-of-speech': 'PROPN', 'lemma': 'Germany'}
```
The feature vector using one hot encoding is:
```
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
````
* **first value: 0**: the feature *lemma* with the value *'s* does not occur in the training instance
* **second value: 0**: the feature *lemma* with the value *European* does not occur in the training instance
* **third value: 1**: the feature *lemma* with the value *Germany* does occur in the training instance
* ...
* **last value: 1**: the feature *part-of-speech* with the value *PROPN* does occur in the training instance


Please note that the number of columns grows immmensely when using one hot encoding (you can easily play with this by changing the input sentence above). Luckily, it is generated automatically by the vectorizer.

### 3 Training an SVM model

Hopefully, you can see that the vectors we end up with here resemble the ones we generated with bag-of-words and tf-idf in the previous notebook. Not surprisingly, we can now use them to train and test a machine learning model, such as SVM. Given that our model is trained on only 7 input vectors, it will not be a meaningful one yet; we will build a model with sufficient data in the assignment.

To train, we also need to have the 'gold' labels for each of the token. Let's define them manually here, according to the example above using the IOB style:

In [ ]:
y=['B-LOC', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG']

Let's now train the model:

In [ ]:
from sklearn import svm

lin_one_hot_clf = svm.LinearSVC()
lin_one_hot_clf.fit(the_array, y)

### 3.1 Testing our model on new examples

We can now reuse the same vectorizer of the training data to vectorize any new example we want to train, and perform prediction on it using the trained SVM model:

In [ ]:
new_training_instances={'lemma': 'America', 'part-of-speech': 'PROPN'}
vectorized=vec.transform(new_training_instances)
print(vectorized.toarray())

In [ ]:
pred=lin_one_hot_clf.predict(vectorized)
pred

Well done! You have now managed to train an SVM model based on two features and apply it to test on some new example to assign named entity annotations automatically. 

Feel free to play around with the training data, the features, and the test data. We will work with more data and more features in the assignment.

Let us finish this section with several key observations:
* Our vectorized representations will easily become too large. For example, only the lemma feature could easily have thousands of values. On the other hand, they are **sparse** representations, containing mostly zeros and occassional 1 values. Is there a better way to encode our data in vectors? The answer is: yes. We will work with **dense** representations in the next notebook!
* In the test example above, the lemma of America was not found in the training data, so no existing lemma received a value of 1 in the final vector representation. This is because the set of feature values is 'frozen' after the training, any new feature value encountered at test time is considered to be *unknown* (typically called **UNK** in NLP terminology).
* Finally, a note on the algorithm. SVM can yield some powerful models if we use good features and train it well, however, it does not have an intrinsic capability to capture **sequences of words**. For this purpose, people often use other algorithms such as Conditional-Random-Fields or a recurrent neural network. You will learn more about this in the machine learning course.

## End of this notebook